In [ ]:
import requests
import time
from pprint import pprint


# ip='45.113.232.43'
ip='localhost'
req_body = {
    "domain":"(define (domain blocksworld) (:requirements :strips) (:predicates (ON ?x ?y) (ontable ?x) (CLEAR ?x) (arm-free) (holding ?x) ) (:action pick-up :parameters (?x) :precondition (and (clear ?x) (ontable ?x) (arm-free)) :effect (and (not (ontable ?x)) (not (clear ?x)) (not (arm-free)) (holding ?x))) (:action put-down :parameters (?x) :precondition (holding ?x) :effect (and (not (holding ?x)) (clear ?x) (arm-free) (ontable ?x))) (:action stack :parameters (?x ?y) :precondition (and (holding ?x) (clear ?y)) :effect (and (not (holding ?x)) (not (clear ?y)) (clear ?x) (arm-free) (on ?x ?y))) (:action unstack :parameters (?x ?y) :precondition (and (on ?x ?y) (clear ?x) (arm-free)) :effect (and (holding ?x) (clear ?y) (not (clear ?x)) (not (arm-free)) (not (on ?x ?y)))))",
    "problem":"(define (problem blocksworld-4-2) (:domain blocksworld) (:objects b d c a ) (:init (clear a) (clear c) (clear d) (ontable a) (ontable b) (ontable d) (on c b) (arm-free)) (:goal (and (on a b) (on b c) (on c d))))"
}

In [ ]:
# An example of Package API to send task and retrive plans.

def test_planner(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body).json()
    celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
    print('Computing...')
    while celery_result.json().get("status","") == 'PENDING':
        time.sleep(2)
        celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])

    pprint(celery_result.json())
    
test_planner("lama",req_body)

In [ ]:
# Function to test the limiter functionality

def test_limiter(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body)
    return solve_request
    

import concurrent.futures
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(30):
        futures.append(executor.submit(test_limiter, "lama",req_body))
    for future in concurrent.futures.as_completed(futures):
        print(future.result())